**DESCRIPTION:**

This Jupyter Notebook shows the data exploration of the 3 files that will be used to create the model that predicts the winner of the World Cup. Then it will proceed to create it. Finally determining the probable winning team of the World Cup.


# 4. Data exploration

**Libraries**

In [1]:
import pandas as pd   # To work with DataFrames
import pickle   # Read binary file
from scipy.stats import poisson   # Poisson probability distribution

## 4.1 Data

### 4.1.1 Groups dictionary of the World Cup Qatar 2022

In [2]:
# Opening a binary file
dict_of_groups = pickle.load(open('Data/Dictionary_of_groups', 'rb'))
dict_of_groups

{'Group A':    Pos         Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1    Qatar (H)    0  0  0  0   0   0   0    0
 1    2      Ecuador    0  0  0  0   0   0   0    0
 2    3      Senegal    0  0  0  0   0   0   0    0
 3    4  Netherlands    0  0  0  0   0   0   0    0,
 'Group B':    Pos           Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1        England    0  0  0  0   0   0   0    0
 1    2           Iran    0  0  0  0   0   0   0    0
 2    3  United States    0  0  0  0   0   0   0    0
 3    4          Wales    0  0  0  0   0   0   0    0,
 'Group C':    Pos          Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1     Argentina    0  0  0  0   0   0   0    0
 1    2  Saudi Arabia    0  0  0  0   0   0   0    0
 2    3        Mexico    0  0  0  0   0   0   0    0
 3    4        Poland    0  0  0  0   0   0   0    0,
 'Group D':    Pos       Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1     France    0  0  0  0   0   0   0    0
 1    2  Australia    0  0  0  0   0   0   0    0
 2    3 

In [3]:
print('The groups are: \n')
for group in dict_of_groups.keys():
    print(f' {group}')

The groups are: 

 Group A
 Group B
 Group C
 Group D
 Group E
 Group F
 Group G
 Group H


In [4]:
# Each group consists of 4 teams. 
#  For example: "Group H" has Portugal, Ghana, Uruguay y South Korea
dict_of_groups['Group H']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Portugal,0,0,0,0,0,0,0,0
1,2,Ghana,0,0,0,0,0,0,0,0
2,3,Uruguay,0,0,0,0,0,0,0,0
3,4,South Korea,0,0,0,0,0,0,0,0


### 4.1.2 Historical data of the Worldcups from 1930 to 2018

In [5]:
df_historical_data = pd.read_csv('Data/Historical_data_clean.csv')
df_historical_data

,FirstTeam,SecondTeam,Year,FirstTeamGoals,SecondTeamGoals
0,France,Mexico,1930,4,1
1,Argentina,France,1930,1,0
2,Chile,Mexico,1930,3,0
3,Chile,France,1930,1,0
4,Argentina,Mexico,1930,6,3
...,...,...,...,...,...
895,Russia,Croatia,2018,2,2
896,France,Belgium,2018,1,0
897,Croatia,England,2018,2,1
898,Belgium,England,2018,2,0


In [6]:
print(f"From 1930 to 2018, there are {df_historical_data.shape[0] + 1} matches.\
 However, one match has been eliminated because it was 'Walkover'")

From 1930 to 2018, there are 901 matches. However, one match has been eliminated because it was 'Walkover'


In [7]:
df_historical_data[['FirstTeamGoals','SecondTeamGoals']].describe().round()

# The maximum number of goals that have been scored in a WorldCup is 10.

,FirstTeamGoals,SecondTeamGoals
count,900.0,900.0
mean,2.0,1.0
std,2.0,1.0
min,0.0,0.0
25%,1.0,0.0
50%,1.0,1.0
75%,3.0,2.0
max,10.0,7.0


### 4.1.3 Qatar 2022 Match Fixture

In [8]:
df_fixture_data = pd.read_csv('Data/Fixture_data_clean.csv', 
                              header=0,
                              names=['FirstTeam', 'Score', 'SecondTeam', 'Year']
                             )
df_fixture_data

,FirstTeam,Score,SecondTeam,Year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


In [9]:
print(f"There are {df_fixture_data.shape[0] + 1} matches in the Qatar 2022 World Cup.")

There are 65 matches in the Qatar 2022 World Cup.


## 4.2 Calculate team strength

In [10]:
# Create DataFrames for the first and second teams 
#  with specific columns: goals scored and conceded.
df_first_team = df_historical_data[['FirstTeam', 'FirstTeamGoals', 'SecondTeamGoals']]
df_second_team = df_historical_data[['SecondTeam', 'SecondTeamGoals', 'FirstTeamGoals']]

In [11]:
# Rename its columns

df_first_team = df_first_team.rename(columns={'FirstTeam':'Team', 
                                              'FirstTeamGoals':'GoalsScored', 
                                              'SecondTeamGoals':'GoalsConceded'}
                                    )

df_second_team = df_second_team.rename(columns={'SecondTeam':'Team', 
                                                'SecondTeamGoals':'GoalsScored', 
                                                'FirstTeamGoals':'GoalsConceded'}
                                      )

In [12]:
df_first_team

,Team,GoalsScored,GoalsConceded
0,France,4,1
1,Argentina,1,0
2,Chile,3,0
3,Chile,1,0
4,Argentina,6,3
...,...,...,...
895,Russia,2,2
896,France,1,0
897,Croatia,2,1
898,Belgium,2,0


In [13]:
df_second_team

,Team,GoalsScored,GoalsConceded
0,Mexico,1,4
1,France,0,1
2,Mexico,0,3
3,France,0,1
4,Mexico,3,6
...,...,...,...
895,Croatia,2,2
896,Belgium,0,1
897,England,1,2
898,England,0,2


In [14]:
# Concatenate both DataFrames (df_first_team & df_second_team)
#  And get the average number of goals conceded and scored by each team.
df_team_strength = pd.concat([df_first_team, df_second_team], 
                             ignore_index=True
                            ).groupby(by='Team').mean()
df_team_strength

# In fact, the STRENGTH OF A TEAM will be given in the context of a match: 
#  (The average of their goals) * (The average of goals received by their opponent).
#  If the opposing team is weak and therefore their goal conceding average is high, 
#  then the other team's average goal value will increase and vice versa.

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


# 5. Modeling

## 5.1 Poisson probability distribution

We use Poisson distribution, because the goal is an event that can occur within 90 minutes of a soccer match.

## $$P(x) = \frac{e^{-\mu}(\mu)^x}{x!}$$

Donde:

$e$ = constante cuyo valor aproximado es 2.718281828459045

$\mu$ = Average goals in 90 minutes between the first team and second team.

$x$ = Number of goals in a match that could be scored by the first team or the second team.

**Library:**

poisson.pmf(k, u)   --->pmf: Probability mass function

## 5.2 Function to predict points

In [15]:
def predict_points(first_team, second_team):
    
    # If the team has already been to a WorldCup
    if first_team in df_team_strength.index and second_team in df_team_strength.index:
        
        # To calculate average goals in a match according to its opponent.
        # goals_scored(Average number of goals scored by a team.) * goals_conceded(Average number of goals conceded by your opponent.)
        mean_first_team = df_team_strength.at[first_team, 'GoalsScored']*df_team_strength.at[second_team, 'GoalsConceded']
        mean_second_team = df_team_strength.at[second_team, 'GoalsScored']*df_team_strength.at[first_team, 'GoalsConceded']
        
        # Probability that the first wins, second team wins or there is a draw.
        prob_first_team, prob_second_team, prob_draw = 0, 0, 0
        
        # All the possible goal results of the matches.
        for x in range(11):   #Number of goals first team between 0 to 10
            for y in range(11):   #Number of goals second team between 0 to 10
                # We use poisson.pmf(x, u) ---> pmf (Probability mass function)
                prob = poisson.pmf(x, mean_first_team) * poisson.pmf(y, mean_second_team)
                if x == y:
                    prob_draw += prob
                elif x > y:
                    prob_first_team += prob
                else:
                    prob_second_team += prob
        
        # The winner of a match gets three points, 
        #  the loser zero point, and both get one point if they draw.
        points_first_team = 3*prob_first_team + 1*prob_draw
        points_second_team = 3*prob_second_team + 1*prob_draw
        
        return (points_first_team, points_second_team)
    
    # Teams that never participated in a worldcup
    else:
        return (0, 0)
    

### 5.2.1 Testing the function

**Argentina VS Mexico**

In [16]:
predict_points('Argentina', 'Mexico')

(2.3129151525530505, 0.5378377125059863)

In [17]:
prob_arg = ((predict_points('Argentina', 'Mexico')[0]/3)*100).round(2)

In [18]:
print(f'Argentina has a {prob_arg}% chance to win.')

Argentina has a 77.1% chance to win.


**England VS United States**

In [19]:
predict_points('England', 'United States')

(2.2356147635326007, 0.5922397535606193)

**Qatar VS Ecuador**

In [20]:
predict_points('Qatar', 'Ecuador')

# We get that result because it's the first time Qatar has participated in a Worldcup.

(0, 0)

## 5.3 Predict the worldcup

In [21]:
# Divide the fixture data
df_fixture_group_48 = df_fixture_data[:48].copy()
df_fixture_knockout = df_fixture_data[48:56].copy()
df_fixture_quarter = df_fixture_data[56:60].copy()
df_fixture_semi = df_fixture_data[60:62].copy()
df_fixture_final = df_fixture_data[62:].copy()

### 5.3.1 Group stage

In [22]:
df_fixture_group_48

,FirstTeam,Score,SecondTeam,Year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
5,Netherlands,Match 36,Qatar,2022
6,England,Match 3,Iran,2022
7,United States,Match 4,Wales,2022
8,Wales,Match 17,Iran,2022
9,England,Match 20,United States,2022


In [23]:
dict_of_groups

{'Group A':    Pos         Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1    Qatar (H)    0  0  0  0   0   0   0    0
 1    2      Ecuador    0  0  0  0   0   0   0    0
 2    3      Senegal    0  0  0  0   0   0   0    0
 3    4  Netherlands    0  0  0  0   0   0   0    0,
 'Group B':    Pos           Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1        England    0  0  0  0   0   0   0    0
 1    2           Iran    0  0  0  0   0   0   0    0
 2    3  United States    0  0  0  0   0   0   0    0
 3    4          Wales    0  0  0  0   0   0   0    0,
 'Group C':    Pos          Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1     Argentina    0  0  0  0   0   0   0    0
 1    2  Saudi Arabia    0  0  0  0   0   0   0    0
 2    3        Mexico    0  0  0  0   0   0   0    0
 3    4        Poland    0  0  0  0   0   0   0    0,
 'Group D':    Pos       Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1     France    0  0  0  0   0   0   0    0
 1    2  Australia    0  0  0  0   0   0   0    0
 2    3 

In [24]:
dict_of_groups['Group A']['Team'].values

array(['Qatar (H)', 'Ecuador', 'Senegal', 'Netherlands'], dtype=object)

In [25]:
for group in dict_of_groups:
    print(dict_of_groups[group]['Team'].values)

['Qatar (H)' 'Ecuador' 'Senegal' 'Netherlands']
['England' 'Iran' 'United States' 'Wales']
['Argentina' 'Saudi Arabia' 'Mexico' 'Poland']
['France' 'Australia' 'Denmark' 'Tunisia']
['Spain' 'Costa Rica' 'Germany' 'Japan']
['Belgium' 'Canada' 'Morocco' 'Croatia']
['Brazil' 'Serbia' 'Switzerland' 'Cameroon']
['Portugal' 'Ghana' 'Uruguay' 'South Korea']


In [26]:
df_fixture_group_48[df_fixture_group_48['FirstTeam'].isin(['Argentina', 'Saudi Arabia', 'Mexico', 'Poland'])]

,FirstTeam,Score,SecondTeam,Year
12,Argentina,Match 8,Saudi Arabia,2022
13,Mexico,Match 7,Poland,2022
14,Poland,Match 22,Saudi Arabia,2022
15,Argentina,Match 24,Mexico,2022
16,Poland,Match 39,Argentina,2022
17,Saudi Arabia,Match 40,Mexico,2022


In [27]:
# Result of the matches

for group in dict_of_groups:   #Group name
    teams_in_group = dict_of_groups[group]['Team'].values   #Teams of each group
    
    # The 6 matches of each group
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['FirstTeam'].isin(teams_in_group)]
    
    # The points of each team by group
    for index, row in df_fixture_group_6.iterrows():
        firstTeam, secondTeam = row['FirstTeam'], row['SecondTeam']
        points_first_team, points_second_team = predict_points(firstTeam, secondTeam)
        
        # Saving the points in the dictionary of the groups.
        dict_of_groups[group].loc[dict_of_groups[group]['Team'] == firstTeam, 'Pts'] += points_first_team
        dict_of_groups[group].loc[dict_of_groups[group]['Team'] == secondTeam, 'Pts'] += points_second_team
    
    # The best two teams for each group
    dict_of_groups[group] = dict_of_groups[group].sort_values('Pts', ascending=False).reset_index()
    dict_of_groups[group] = dict_of_groups[group][['Team','Pts']]
    dict_of_groups[group] = dict_of_groups[group].round()

In [28]:
dict_of_groups

{'Group A':           Team  Pts
 0  Netherlands  4.0
 1      Senegal  2.0
 2      Ecuador  2.0
 3    Qatar (H)  0.0,
 'Group B':             Team  Pts
 0        England  6.0
 1          Wales  5.0
 2  United States  3.0
 3           Iran  2.0,
 'Group C':            Team  Pts
 0     Argentina  7.0
 1        Poland  6.0
 2        Mexico  4.0
 3  Saudi Arabia  1.0,
 'Group D':         Team  Pts
 0     France  7.0
 1    Denmark  6.0
 2    Tunisia  3.0
 3  Australia  2.0,
 'Group E':          Team  Pts
 0     Germany  7.0
 1       Spain  5.0
 2       Japan  3.0
 3  Costa Rica  2.0,
 'Group F':       Team  Pts
 0  Croatia  7.0
 1  Belgium  6.0
 2  Morocco  4.0
 3   Canada  0.0,
 'Group G':           Team  Pts
 0       Brazil  8.0
 1  Switzerland  4.0
 2       Serbia  3.0
 3     Cameroon  2.0,
 'Group H':           Team  Pts
 0     Portugal  6.0
 1      Uruguay  5.0
 2        Ghana  4.0
 3  South Korea  2.0}

In [29]:
# Portugal and Uruguay pass in group H
dict_of_groups['Group H']

,Team,Pts
0,Portugal,6.0
1,Uruguay,5.0
2,Ghana,4.0
3,South Korea,2.0


### 5.3.2 Knockout

In [30]:
df_fixture_knockout

,FirstTeam,Score,SecondTeam,Year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [31]:
# Update of the DataFrame "df_fixture_knockout" with the teams that managed 
#  to pass from dict_of_groups

for group in dict_of_groups:
    group_winner = dict_of_groups[group].loc[0, 'Team']   # First row and "Team" column
    group_runnerUp = dict_of_groups[group].loc[1, 'Team']   # Second row and "Team" column
    df_fixture_knockout.replace({f'Winners {group}': group_winner, 
                                 f'Runners-up {group}': group_runnerUp}, 
                                inplace=True
                               )
    
df_fixture_knockout['Winner'] = '?'   # New column for the winner

In [32]:
df_fixture_knockout

,FirstTeam,Score,SecondTeam,Year,Winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [33]:
# Function to get the winner

def get_winner(df_fixture_updated):
    
    for index, row in df_fixture_updated.iterrows():
        firstTeam, secondTeam = row['FirstTeam'], row['SecondTeam']
        
        # Apply the "predict_points" function to define the winner
        points_first_team, points_second_team = predict_points(firstTeam, secondTeam)
        
        if points_first_team > points_second_team:
            winner = firstTeam
        else:
            winner = secondTeam
            
        df_fixture_updated.loc[index, 'Winner'] = winner
    
    return df_fixture_updated

In [34]:
# Get the teams that pass to the quarterfinals.
get_winner(df_fixture_knockout)

,FirstTeam,Score,SecondTeam,Year,Winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


### 5.3.3 Quarterfinal

In [35]:
df_fixture_quarter

,FirstTeam,Score,SecondTeam,Year
56,Winners Match 53,Match 58,Winners Match 54,2022
57,Winners Match 49,Match 57,Winners Match 50,2022
58,Winners Match 55,Match 60,Winners Match 56,2022
59,Winners Match 51,Match 59,Winners Match 52,2022


In [36]:
# Create function to update DataFrame "df_fixture_quarter", "df_fixture_semi" 
#  from previous DataFrame

def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner_team = df_fixture_round_1.loc[index, 'Winner']   # Get the winner team
        match_number = df_fixture_round_1.loc[index, 'Score']   # Get the match number
        
        # Replace the winner team according to the match number.
        df_fixture_round_2.replace({f'Winners {match_number}':winner_team}, inplace=True)
    
    # Create a new result column
    df_fixture_round_2['Winner'] = '?'
    
    return df_fixture_round_2

In [37]:
# Update of the DataFrame "df_fixture_quarter" with the teams that managed to pass.
update_table(df_fixture_knockout, df_fixture_quarter)

,FirstTeam,Score,SecondTeam,Year,Winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [38]:
# Get the teams that pass to the semifinal.
get_winner(df_fixture_quarter)

,FirstTeam,Score,SecondTeam,Year,Winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


### 5.3.4 Semifinal

In [39]:
df_fixture_semi

,FirstTeam,Score,SecondTeam,Year
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022


In [40]:
# Update of the DataFrame "df_fixture_semi" with the teams that managed to pass.
update_table(df_fixture_quarter, df_fixture_semi)

,FirstTeam,Score,SecondTeam,Year,Winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [41]:
# Get the teams that pass to the final.
get_winner(df_fixture_semi)

,FirstTeam,Score,SecondTeam,Year,Winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


### 5.3.5 Final

In [42]:
df_fixture_final

,FirstTeam,Score,SecondTeam,Year
62,Losers Match 61,Match 63,Losers Match 62,2022
63,Winners Match 61,Match 64,Winners Match 62,2022


In [43]:
# Update of the DataFrame "df_fixture_final" with the teams that managed to pass.
update_table(df_fixture_semi, df_fixture_final)

,FirstTeam,Score,SecondTeam,Year,Winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [44]:
# Get the worldcup winner team
get_winner(df_fixture_final)

,FirstTeam,Score,SecondTeam,Year,Winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil


In [45]:
winner_team = df_fixture_final.loc[63]['Winner']

In [46]:
print(f'Worldcup Winning Team is {winner_team}.')

Worldcup Winning Team is Brazil.


**Since we know the winner of the World Cup, let's calculate the number of goals with which team wins.**

In [47]:
# The strength of the Brazil team based on its rival France.
Brazil_strength = df_team_strength.at['Brazil', 'GoalsScored']*df_team_strength.at['France', 'GoalsConceded']

for goal in range(11):
    prob = poisson.pmf(goal, Brazil_strength)
    prob = (prob*100).round(2)
    print(f'The probability that Brazil wins with {goal} goals is {prob}%')

The probability that Brazil wins with 0 goals is 8.62%
The probability that Brazil wins with 1 goals is 21.13%
The probability that Brazil wins with 2 goals is 25.89%
The probability that Brazil wins with 3 goals is 21.16%
The probability that Brazil wins with 4 goals is 12.96%
The probability that Brazil wins with 5 goals is 6.35%
The probability that Brazil wins with 6 goals is 2.6%
The probability that Brazil wins with 7 goals is 0.91%
The probability that Brazil wins with 8 goals is 0.28%
The probability that Brazil wins with 9 goals is 0.08%
The probability that Brazil wins with 10 goals is 0.02%


In [48]:
predict_points('Brazil', 'France')

(1.793819490591848, 1.0145278238178859)

In [49]:
# Probability(%) that Brazil will win
((predict_points('Brazil', 'France')[0]/3)*100).round(2)

59.79

In [50]:
# Probability(%) that France will win
((predict_points('Brazil', 'France')[1]/3)*100).round(2)

33.82

## 5.4 Conclusion:

The World Cup winning team is Brazil with a probability of 59.79% and with a higher probability of 25.89% of scoring 2 goals against their opponent France.